### Script to detect all cameras available and there FPS for different resolutions

In [2]:
import cv2
import time

def list_cameras(max_index=10):
    available = []
    for i in range(max_index):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available.append(i)
            cap.release()
    return available

def measure_fps(camera_index, width, height, test_seconds=3,target_fps=60):
    # Open via V4L2 backend (default on Linux)
    cap = cv2.VideoCapture(camera_index, cv2.CAP_V4L2)

    # Explicitly request MJPEG compression and 60 FPS
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    cap.set(cv2.CAP_PROP_FPS, target_fps)

    if not cap.isOpened():
        print(f"❌ Camera {camera_index} could not be opened.")
        return None

    frame_count = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if (time.time() - start_time) >= test_seconds:
            break

    elapsed = time.time() - start_time
    cap.release()

    if elapsed == 0:
        return None

    return frame_count / elapsed

def test_all_cameras():
    cameras = list_cameras()
    if not cameras:
        print("❌ No cameras detected.")
        return

    print(f"✅ Detected cameras: {cameras}")

    resolutions = [
        (640, 480),
        (1280, 720),
        (1920, 1080)
    ]

    results = {}

    for cam_index in cameras:
        print(f"\n=== Testing Camera {cam_index} ===")
        results[cam_index] = {}
        for (w, h) in resolutions:
            fps = measure_fps(cam_index, w, h)
            if fps is not None:
                results[cam_index][f"{w}x{h}"] = round(fps, 2)
                print(f"Resolution {w}x{h}: {fps:.2f} FPS")
            else:
                results[cam_index][f"{w}x{h}"] = "N/A"
                print(f"Resolution {w}x{h}: N/A")

    print("\n========== SUMMARY ==========")
    for cam_index, res_data in results.items():
        print(f"\nCamera {cam_index}:")
        for res, fps in res_data.items():
            print(f"  {res:<10} → {fps}")

if __name__ == "__main__":
    test_all_cameras()


[ WARN:0@78.897] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@79.132] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
[ WARN:0@79.163] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@79.168] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
[ WARN:0@79.200] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@79.204] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
[ WARN:0@79.204] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@79.208] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
[ WARN:0@79.208] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video7): can't open camera by index
[ERROR:0@79.212] global obsensor_uvc_stream_channel.cpp:163 getStreamChan

✅ Detected cameras: [0, 2, 4]

=== Testing Camera 0 ===
Resolution 640x480: 27.82 FPS
Resolution 1280x720: 27.93 FPS
Resolution 1920x1080: 27.82 FPS

=== Testing Camera 2 ===
Resolution 640x480: 53.63 FPS
Resolution 1280x720: 54.33 FPS
Resolution 1920x1080: 53.92 FPS

=== Testing Camera 4 ===
Resolution 640x480: 32.67 FPS
Resolution 1280x720: 32.18 FPS
Resolution 1920x1080: 31.83 FPS

========== SUMMARY ==========

Camera 0:
  640x480    → 27.82
  1280x720   → 27.93
  1920x1080  → 27.82

Camera 2:
  640x480    → 53.63
  1280x720   → 54.33
  1920x1080  → 53.92

Camera 4:
  640x480    → 32.67
  1280x720   → 32.18
  1920x1080  → 31.83


## Capture Images: 
click space bar to capture images.
1) Download and print any OpenCV calibration checkerboard on A4 paper.
Use this standard 9×6 inner corner pattern (i.e. 10×7 squares):
https://github.com/opencv/opencv/blob/master/doc/pattern.png

2) Print it without scaling (100% size).
Each square should be 25 mm if printed properly — but you can use any consistent size; you’ll specify the square size in meters later. eg: got 23.5 mm as square size in my case after printing.

3) Stick the sheet on a flat board (not wrinkled).

4) Capture 15–25 images of the checkerboard at different angles, distances, and positions.

Make sure:

* the entire board is visible in each shot,

* you tilt it up/down/sideways (don’t keep it flat),

* lighting is bright (no glare or blur).

In [5]:
import cv2
import time
import os

# Replace with your phone IP
def select_camera():
    print("\n=== Camera Selection ===")
    print("1. Kreo Webcam #1 (USB index 0)")
    print("2. Kreo Webcam #2 (USB index 4)")
    print("3. Mobile IP Webcam (enter URL manually)")
    choice = input("Select camera [1/2/3]: ").strip()

    if choice == "1":
        src = 2
        folder = "../data/calibration_kreo1"
    elif choice == "2":
        src = 4
        folder = "../data/calibration_kreo2"
    elif choice == "3":
        ip_cam_url = input("Enter IP Webcam URL (e.g., http://192.168.x.x:8080/video): ").strip()
        src = ip_cam_url
        folder = "../../data/mobile_calibration"
    else:
        print("Invalid choice. Exiting.")
        exit(1)

    os.makedirs(folder, exist_ok=True)
    return src, folder

src, folder = select_camera()

print(f"\n[INFO] Opening camera source: {src}")
cap = cv2.VideoCapture(src, cv2.CAP_V4L2)

if not cap.isOpened():
    raise RuntimeError(f"Cannot open camera source: {src}")

# Try setting resolution for USB webcams
if isinstance(src, int):
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 60)
    
i = 0
print("\nPress SPACE to capture a frame for calibration.")
print("Press ESC to exit.\n")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[WARN] Frame grab failed, retrying...")
        time.sleep(0.1)
        continue

    # Overlay instructions and camera info
    res_text = f"RES: {frame.shape[1]}x{frame.shape[0]}"
    cv2.putText(frame, res_text, (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.putText(frame, "Press SPACE to save frame, ESC to quit.",
                (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    cv2.imshow("Calibration Capture", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == 32:  # SPACE
        fname = os.path.join(folder, f"calib_{i:02d}.jpg")
        cv2.imwrite(fname, frame)
        print(f"[INFO] Saved {fname}")
        i += 1
        time.sleep(0.3)

    elif key == 27:  # ESC
        print("[INFO] Exiting capture session.")
        break

cap.release()
cv2.destroyAllWindows()



=== Camera Selection ===
1. Kreo Webcam #1 (USB index 0)
2. Kreo Webcam #2 (USB index 4)
3. Mobile IP Webcam (enter URL manually)

[INFO] Opening camera source: 2

Press SPACE to capture a frame for calibration.
Press ESC to exit.

[INFO] Saved ../data/calibration_kreo1/calib_00.jpg
[INFO] Exiting capture session.


## Getting Intrinsic Values of Camera

run the below script to get the Intrinsic values of your camera using the images captured by the above script.

In [7]:
import cv2
import numpy as np
import glob
import math
import os

# ==================== CAMERA SELECTION ====================
print("\n=== Camera Calibration Selection ===")
print("1. Kreo Webcam #1 (data/calibration_kreo1)")
print("2. Kreo Webcam #2 (data/calibration_kreo2)")
print("3. Mobile IP Webcam (data/calibration_mobile)")
choice = input("Select camera [1/2/3]: ").strip()

if choice == "1":
    cam_name = "kreo1"
    folder = "../../data/calibration_kreo1"
elif choice == "2":
    cam_name = "kreo2"
    folder = "../../data/calibration_kreo2"
elif choice == "3":
    cam_name = "mobile"
    folder = "../../data/calibration_mobile"
else:
    print("[ERROR] Invalid choice. Exiting.")
    exit(1)

pattern = os.path.join(folder, "calib_*.jpg")
images = sorted(glob.glob(pattern))
if not images:
    print(f"[ERROR] No images found in {folder}")
    exit(1)

print(f"[INFO] Found {len(images)} images for calibration in {folder}")

# ==================== PARAMETERS ====================
CHECKERBOARD = (9, 6)
SQUARE_SIZE = 0.0235  # meters (23.5 mm per square)

objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE

objpoints = []
imgpoints = []

# ==================== CORNER DETECTION ====================
for i, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(
            gray, corners, (11, 11), (-1, -1),
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        imgpoints.append(corners2)
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        cv2.imshow("Corners", img)
        cv2.waitKey(80)
    else:
        print(f"[WARN] Chessboard not found in {os.path.basename(fname)}")

cv2.destroyAllWindows()

# ==================== INITIAL CALIBRATION ====================
print("\n[INFO] Running initial calibration ...")
ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

if not ret:
    print("[ERROR] Calibration failed.")
    exit(1)

print("\n[RESULT] Camera calibrated successfully!")
print("Camera matrix:\n", cameraMatrix)
print("Distortion coefficients:\n", distCoeffs.ravel())



=== Camera Calibration Selection ===
1. Kreo Webcam #1 (data/calibration_kreo1)
2. Kreo Webcam #2 (data/calibration_kreo2)
3. Mobile IP Webcam (data/calibration_mobile)
[INFO] Found 71 images for calibration in ../../data/calibration_kreo2

[INFO] Running initial calibration ...

[RESULT] Camera calibrated successfully!
Camera matrix:
 [[1.03213061e+03 0.00000000e+00 6.21544506e+02]
 [0.00000000e+00 1.03522598e+03 3.14798879e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [ 0.0610174  -0.13275463 -0.01383168 -0.00186799  0.06734642]


## Check Calibration

check the RMSE of Calibration:
* ```RMSE < 0.5 px``` = excellent

* ```0.5–1.0 px``` = good

* ```1.0–2.0 px``` = acceptable but you might see small metric errors

* ```> 2.0 px``` = calibration likely poor — redo

If your RMSE is > ~1.0 px, follow the cleanup steps present at the end.

In [8]:

# ==================== REPROJECTION ERROR ANALYSIS ====================
print("\n[INFO] Computing per-image reprojection errors...")
per_image_errs = []
bad = []
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs)
    err = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    per_image_errs.append(err)
    fname = os.path.basename(images[i])
    print(f"{fname:30s} RMSE = {err:.4f}")
    if err > 1.5:
        bad.append(images[i])

rmse = math.sqrt(sum(e*e for e in per_image_errs) / len(per_image_errs))
print(f"\n[INFO] Overall RMSE = {rmse:.4f} px")
print(f"[INFO] Found {len(bad)} images with error > 1.5 px")



[INFO] Computing per-image reprojection errors...
calib_00.jpg                   RMSE = 0.1020
calib_01.jpg                   RMSE = 0.0521
calib_02.jpg                   RMSE = 0.0289
calib_03.jpg                   RMSE = 0.0190
calib_04.jpg                   RMSE = 0.0193
calib_05.jpg                   RMSE = 0.0139
calib_06.jpg                   RMSE = 0.0168
calib_07.jpg                   RMSE = 0.1025
calib_08.jpg                   RMSE = 0.0700
calib_09.jpg                   RMSE = 0.0517
calib_10.jpg                   RMSE = 0.0463
calib_11.jpg                   RMSE = 0.0394
calib_12.jpg                   RMSE = 0.0664
calib_13.jpg                   RMSE = 0.1009
calib_14.jpg                   RMSE = 0.0713
calib_15.jpg                   RMSE = 0.0576
calib_16.jpg                   RMSE = 0.0495
calib_17.jpg                   RMSE = 0.0467
calib_18.jpg                   RMSE = 0.0424
calib_19.jpg                   RMSE = 0.0370
calib_20.jpg                   RMSE = 0.0342
cali

## Removing outliers

### If reprojection error is high — how to fix it (do these, in order)

1) Use more frames (20–40 good ones) — varied angles & distances. Many low-quality frames ruin calibration more than help. Exclude blurry or partially occluded frames.

2) Cover the full FOV — make sure chessboard corners are in the corners and edges of the image in some frames; not always centered.

3) Vary distances — some close, some far.

4) Avoid rolling shutter blur / motion blur — use stills or pause the camera when capturing. 

5) Accurate square size — ensure square size in SQUARE_SIZE exactly matches printed paper squares. 

6) Remove outliers — rerun calibration excluding images with large individual reprojection errors (the script below prints per-image warnings).

7) Use cv2.calibrateCameraExtended — gives per-image reprojection residuals to find bad images.

8) Use fisheye model if lens is very wide-angle (OpenCV has fisheye module)

In [9]:

# ==================== REMOVE OUTLIERS AND RE-RUN ====================
if len(bad) > 0:
    print("\n[INFO] Removing high-error images and recalibrating ...")
    with open(f"bad_images_{cam_name}.txt", "w") as f:
        for b in bad: f.write(b + "\n")

    # filter objpoints/imgpoints
    good_obj = []
    good_img = []
    good_files = []
    for i, err in enumerate(per_image_errs):
        if err <= 1.5:
            good_obj.append(objpoints[i])
            good_img.append(imgpoints[i])
            good_files.append(images[i])

    print(f"[INFO] Using {len(good_obj)} good images for refined calibration.")

    ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
        good_obj, good_img, gray.shape[::-1], None, None
    )

    per_image_errs2 = []
    for i in range(len(good_obj)):
        imgpoints2, _ = cv2.projectPoints(good_obj[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs)
        err = cv2.norm(good_img[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        per_image_errs2.append(err)
    rmse2 = math.sqrt(sum(e*e for e in per_image_errs2) / len(per_image_errs2))

    print(f"\n[INFO] Refined RMSE after removing outliers = {rmse2:.4f} px")
else:
    rmse2 = rmse


### Saving the results

In [10]:

# ==================== SAVE RESULTS ====================
save_name = f"camera_calibration_{cam_name}.npz"
np.savez(save_name, cameraMatrix=cameraMatrix, distCoeffs=distCoeffs)
print(f"\n[SAVED] Intrinsics written to {save_name}")
print("Camera matrix:\n", cameraMatrix)
print("Distortion coefficients:\n", distCoeffs.ravel())

# ==================== QUALITY REPORT ====================
if rmse2 < 0.5:
    quality = "Excellent ✅"
elif rmse2 < 1.0:
    quality = "Good 👍"
else:
    quality = "Acceptable ⚠️"
print(f"\n[QUALITY] Calibration quality: {quality}")
print("[INFO] Done.")



[SAVED] Intrinsics written to camera_calibration_kreo2.npz
Camera matrix:
 [[1.03213061e+03 0.00000000e+00 6.21544506e+02]
 [0.00000000e+00 1.03522598e+03 3.14798879e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [ 0.0610174  -0.13275463 -0.01383168 -0.00186799  0.06734642]

[QUALITY] Calibration quality: Excellent ✅
[INFO] Done.
